##### ### The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2025 Semester 1

## Assignment 1: Scam detection with naive Bayes


**Student ID(s):**     `1462474`


This iPython notebook is a template which you will use for your Assignment 1 submission.

**NOTE: YOU SHOULD ADD YOUR RESULTS, GRAPHS, AND FIGURES FROM YOUR OBSERVATIONS IN THIS FILE TO YOUR REPORT (the PDF file).** Results, figures, etc. which appear in this file but are NOT included in your report will not be marked.

**Adding proper comments to your code is MANDATORY. **

## 1. Supervised model training


In [1]:
## Import necessary libraries
import numpy as np
import math
import pandas as pd

#### Read in supervised training dataset

In [2]:
sms_df = pd.read_csv('sms_supervised_train.csv')

#### Reformat text to help with tokenising

In [3]:
# Ensures data types are as intended and no nulls
def preprocess_data(df):
    df['textPreprocessed'] = df['textPreprocessed'].astype(str)
    df = df.dropna(subset=['textPreprocessed'])
    df['class'] = df['class'].astype(int)
    return df

sms_df = preprocess_data(sms_df)

#### Build vocabulary list

In [4]:
# Define vocab list (set for build efficiency)
vocab_set = set()

# For each row in the dataset, split text into words and add them to the vocab set
# Convert to a list at the end
def create_vocab_list(df):
    vocab_set = set()
    for text in df['textPreprocessed']:
        words = text.split()
        vocab_set.update(words)
    return list(vocab_set)

# Create vocab list
vocab_list = create_vocab_list(sms_df)


#### Build count matrix

In [5]:
# Creates a count matrix where each row represents a text instance
# Columns represent the words in the vocab list.
# Returns the matrix and a dictionary mapping words to their index in the vocab list
def build_count_matrix(df, vocab_list):
    # Initialise empty count matrix
    count_matrix = np.zeros((df.shape[0], len(vocab_list)))
    # Create dictionary mapping words to their index in the vocab list
    vocab_dict = {word: i for i, word in enumerate(vocab_list)}
    for index, text in df['textPreprocessed'].items():
        for word in text.split():
            # If the word is in the vocab list, increment its count in the matrix
            if word in vocab_dict:
                word_index = vocab_dict.get(word)
                count_matrix[index][word_index] += 1
    return count_matrix, vocab_dict

# Build count matrix
count_matrix,vocab_dict = build_count_matrix(sms_df, vocab_list)

#### Compute the prior probability of each class:


In [6]:
# Calculate the prior probability of each class given the dataframe
def calculate_priors(df):
    n_rows = df.shape[0]
    # Count the number of instances in each class
    n_c0 = df[df['class'] == 0].shape[0]
    n_c1 = df[df['class'] == 1].shape[0]
    # Calculate the prior probability of each class
    p_c0 = n_c0 / n_rows
    p_c1 = n_c1 / n_rows
    return [p_c0, p_c1]

# Calculate priors
priors = calculate_priors(sms_df)
p_c0 = priors[0]
p_c1 = priors[1]

# Answer to question 1 in our report
print(f"Our two priors are p_c0 = {p_c0}\nand p_c1 = {p_c1} ")


Our two priors are p_c0 = 0.8
and p_c1 = 0.2 


#### Find the probability of each word appearing in a message from each class (likelihood)

In [7]:
# First we need to find the number of times each word appears in each class

''' Counts the number of times each word appears in a given class
    returns an list of counts '''
def count_words_in_class(class_num, train_df, count_matrix):
    # Find the indexes of the rows in the count matrix that belong to the given class
    index_list = np.where(train_df['class'] == class_num)[0]
    # Get the count matrix for the given class (only shows rows where we classify as class_num)
    class_count_matrix = count_matrix[index_list]
    # Sums the word counts across all text instances for the class
    count_list = class_count_matrix.sum(axis=0)
    return count_list

# Class 0 count
count_c0 = count_words_in_class(0, sms_df, count_matrix)

# Class 1 count
count_c1 = count_words_in_class(1, sms_df, count_matrix)


In [8]:
# We will use laplace smoothing to ensure every event has a non-zero probability
# Since we have sparse data (more on report)

# Laplace smoothing value
alpha = 1

# Laplace function returns the conditional probability of a word given a class
# with laplace smoothing
def laplace(count, alpha, total, v):
    return (count+alpha) / (total + v*alpha)

# Calculates the likelihoods p_(c,i) of word i appearing in a given class c
# returns the probability lists for each word in the vocab list and each class
def calculate_likelihoods(count_c0, count_c1, vocab_dict, alpha=1):
    # Initialise empty lists to store the likelihoods
    class_0_c = []
    class_1_c = []
    # Calculate total count of words in each class
    total_c0 = count_c0.sum()
    total_c1 = count_c1.sum()
    # Find the length of the vocab list
    V = len(vocab_dict)
    # For each class calculate the likelihood list
    for word, index in vocab_dict.items():
        count = count_c0[index]
        # Find prob using laplace smoothing
        p_ci = laplace(count, alpha, total_c0, V)
        class_0_c.append(p_ci)
    for word, index in vocab_dict.items():
        count = count_c1[index]
        # Find prob using laplace smoothing
        p_ci = laplace(count, alpha, total_c1, V)
        class_1_c.append(p_ci)
    return class_0_c, class_1_c

# Find the likelihoods for each class
class_0_c, class_1_c = calculate_likelihoods(count_c0, count_c1, vocab_dict, alpha)


# Check probabilities sum to roughly one
print(sum(class_1_c))
print(sum(class_0_c))


0.999999999999977
1.0000000000000044


### Further supervised model training questions for report

In [9]:
# Question 2
# Find the most probable words in each class

# We find the 10 most probable words by sorting the probabilities list
# and finding their indexes in the original vocab list

'''Finds and returns the n most probable words in a given class
    returns a list in order of probability (descending) and a list
     containing their probability values'''
def find_n_most_probable(n, class_num):
    # Find the count list for the given class
    if class_num == 0:
        prob_list = class_0_c
    elif class_num == 1:
        prob_list = class_1_c
    else:
        return

    # Sort the list and get the indexes of the n most probable words
    sorted_indexes = np.argsort(prob_list)[-n:][::-1]

    sorted_probs = np.sort(prob_list)[-n:][::-1]

    # Get the words from our vocab list using the indexes
    most_probable_words = [vocab_list[i] for i in sorted_indexes]

    return most_probable_words, sorted_probs


# Find the 10 most probable words in class 0
print("Most probable words in class 0:", find_n_most_probable(10, 0)[0])

# List their probability values
print("Probability values for class 0:", find_n_most_probable(10, 0)[1])

# Find the 10 most probable words in class 1
print("Most probable words in class 1:", find_n_most_probable(10, 1)[0])
print("Probability values for class 1:", find_n_most_probable(10, 1)[1])

Most probable words in class 0: ['.', ',', '?', 'u', '...', '!', '..', ';', '&', 'go']
Probability values for class 0: [0.07930378 0.02602418 0.02557645 0.0189165  0.0187486  0.01718155
 0.01494291 0.013152   0.01309604 0.01113723]
Most probable words in class 1: ['.', '!', ',', 'call', '£', 'free', '/', '2', '&', '?']
Probability values for class 1: [0.05652174 0.02434783 0.02347826 0.02054348 0.01391304 0.01054348
 0.00913043 0.00880435 0.00869565 0.00847826]


#### Question 3: Finding the most predictive words


In [10]:
# Calculate the probability ratio of a word occurring in class 0 vs class 1
class_0_c = np.array(class_0_c)
class_1_c = np.array(class_1_c)
p_ratio_non_mal = class_0_c/class_1_c

# We don't have to worry about dividing by zero as we have already applied laplace smoothing

# Find the 10 most strongly predictive words of the non-malicious class
sorted_c0_div_c1 = np.argsort(p_ratio_non_mal)[-10:][::-1]
most_predictive_words_non_mal = [vocab_list[i] for i in sorted_c0_div_c1]
non_scam_ratios = np.sort(p_ratio_non_mal)[-10:][::-1]

# Find the 10 most strongly predictive words of the malicious class
p_ratio_mal = class_1_c/class_0_c
sorted_c1_div_c0 = np.argsort(p_ratio_mal)[-10:][::-1]
most_predictive_words_mal = [vocab_list[i] for i in sorted_c1_div_c0]
scam_ratios = np.sort(p_ratio_mal)[-10:][::-1]

# Print the most predictive words of the non-malicious class
print("The most predictive words of the non-malicious class are:", most_predictive_words_non_mal)
print("The associated probability ratios are:", non_scam_ratios)



# Print the most predictive words of the malicious class
print("The most predictive words of the malicious class are:", most_predictive_words_mal)
print("The associated probability ratios are:", scam_ratios)




The most predictive words of the non-malicious class are: [';', '...', 'gt', 'lt', ':)', 'ü', 'lor', 'hope', 'ok', 'd']
The associated probability ratios are: [60.49921648 57.49570928 54.06312962 53.54824267 47.88448623 31.92299082
 28.83366913 24.71457354 24.71457354 21.1103649 ]
The most predictive words of the malicious class are: ['prize', 'tone', '£', 'select', 'claim', 'paytm', 'code', 'award', 'won', '18']
The associated probability ratios are: [99.05086957 64.09173913 49.71965217 46.61217391 45.96478261 36.90130435
 34.95913043 32.04586957 31.07478261 29.1326087 ]


## 2. Supervised model evaluation

### Predicting the labels of our test set

In [11]:
# Read in our test dataset
test_df = pd.read_csv('sms_test.csv')


In [12]:
# Apply the same preprocessing steps as before
# Ensure all values in the column are strings
test_df['textPreprocessed'] = test_df['textPreprocessed'].astype(str)

# Ensure no null values affect tokenising
# Find num rows
test_rows = test_df.shape[0]
print("Number of entries before dropping null values: ", test_rows)
test_df = test_df.dropna(subset=['textPreprocessed'])
print("Number of entries before after dropping null values: ", test_rows)


Number of entries before dropping null values:  1000
Number of entries before after dropping null values:  1000


In [13]:
# For each test instance, compute a count vector
# which represents the number of times each word in the vocab list appears
# This will be in the form of an N_test x V matrix
def build_test_count_matrix(test_df, vocab_list, vocab_dict):
    # Initialise empty count matrix
    test_count_matrix = np.zeros((test_df.shape[0], len(vocab_list)))
    # Counters and sets for question 2
    unique_words_in_vocab = set()
    unique_words_not_in_vocab = set()
    num_test_words_in_vocab = 0
    num_test_words_not_in_vocab = 0

    # Add words that exist in our vocab to the new count matrix for each row in the data
    for index, text in enumerate(test_df['textPreprocessed']):
        for word in text.split():
            if word in vocab_dict:
                word_index = vocab_dict.get(word)
                test_count_matrix[index][word_index] += 1
                unique_words_in_vocab.add(word)
                num_test_words_in_vocab += 1
            else:
                unique_words_not_in_vocab.add(word)
                num_test_words_not_in_vocab += 1

    return test_count_matrix, num_test_words_in_vocab, num_test_words_not_in_vocab, unique_words_in_vocab, unique_words_not_in_vocab



# Build the test count matrix
test_count_matrix, num_test_words_in_vocab, num_test_words_not_in_vocab, unique_words_in_vocab, unique_words_not_in_vocab = build_test_count_matrix(test_df, vocab_list, vocab_dict)



# Count the number of unique words and in the vocab list and test (for question 2)
num_unique_words_in_vocab = len(unique_words_in_vocab)
num_unique_words_not_in_vocab = len(unique_words_not_in_vocab)
num_test_words_total = num_test_words_in_vocab + num_test_words_not_in_vocab




In [14]:
# Check if all test instances contain words from the vocab list
# If an instance contains no words from the vocab list, we won't classify it

for row in test_count_matrix:
    if row.sum() == 0:
        print("Found a row with no words from the vocab list")
        print(row)



No test items skipped as this code does not output any rows (refer to q2 below)

#### Compute the posterior probability of each class given the observed word count

In [15]:
# For each test instance, we compute the log posterior probability of observing each
# word given the class label. Returns a list of log posterior probabilities for each class and probabilities
def calc_posterior_log(count_matrix, likelihoods_list, label_index, priors):
    # Initialise empty list to store posterior probabilities
    posterior_probs = []
    log_posterior_probs = []
    prior_value = priors[label_index]

    # For each test instance
    for row in count_matrix:
        # Initialise posterior probability
        posterior = np.log(prior_value)

        # For each word in the vocab list
        for i, word_freq in enumerate(row):
            if word_freq > 0:
                # Get the likelihood of the word given the class label
                likelihood_value = likelihoods_list[i]

                # Take the log of this value to combat overflow
                log_likelihood_value = np.log(likelihood_value)


                # Update the posterior probability
                posterior += (log_likelihood_value * word_freq)


        # Find the log multinomial coefficient
        # Using log gamma function to avoid overflow
        log_multinomial_coefficient = (
            math.lgamma(row.sum() + 1) - np.sum([math.lgamma(c + 1) for c in row])
        )

        posterior += log_multinomial_coefficient

        # Add the posterior probability to our list
        log_posterior_probs.append(posterior)
        # Convert log posterior to normal posterior
        posterior_probs.append(np.exp(posterior))

    return log_posterior_probs, posterior_probs

# Class 0 posterior
log_c0_post, c0_posterior = calc_posterior_log(test_count_matrix, class_0_c, 0, priors)

# Class 1 posterior

log_c1_post, c1_posterior = calc_posterior_log(test_count_matrix, class_1_c, 1, priors)


print(max(c1_posterior))


0.011304347826086959


#### Predict the labels of each test instance

In [16]:
''' Predicts the labels of each test instance using the log posterior probabilities '''
def predict_nb(c0_posterior, c1_posterior):

    # Using our previously calculated log posterior probabilities:

    # Find argmax for each instance
    argmax_labels = []
    associated_probs = []

    for post_prob_0, post_prob_1 in zip(c0_posterior, c1_posterior):
        # Find the maximum posterior probability
        if post_prob_0 >= post_prob_1:
            associated_probs.append(post_prob_0)
            argmax_labels.append(0)

        # Otherwise if probabilities are less we classify as 1
        else:
            associated_probs.append(post_prob_1)
            argmax_labels.append(1)

    return argmax_labels, associated_probs


# Run on our dataset
pred_labels, probs = predict_nb(log_c0_post, log_c1_post)

print(pred_labels[1:20])



[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]


#### Find the accuracy of our model (Q1)

In [17]:
# Overall accuracy of our classifier

def get_accuracy(labels, true_labels):
    # Find the number of correct predictions
    num_correct = np.sum(labels == true_labels)

    # Calculate the accuracy
    accuracy = num_correct / len(true_labels)

    return accuracy

# Get the true labels
true_labels = test_df['class'].values

# Prints the accuracy, confusion matrix, precision and recall
# of our model's predictions
def model_eval(true_labels, pred_labels):
    accuracy = get_accuracy(true_labels, pred_labels)
    print("Accuracy:", accuracy)

    # Confusion matrix
    confusion_matrix = np.zeros((2, 2), dtype=int)
    # Fill the confusion matrix
    for true, predicted in zip(true_labels, pred_labels):
        confusion_matrix[true][predicted] += 1
    print("Confusion Matrix:\n", confusion_matrix)

    TN = confusion_matrix[0, 0]
    FP = confusion_matrix[0, 1]
    FN = confusion_matrix[1, 0]
    TP = confusion_matrix[1, 1]

    # Calculate precision and recall
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")

# Evaluate our model
model_eval(true_labels, pred_labels)

Accuracy: 0.975
Confusion Matrix:
 [[785  15]
 [ 10 190]]
Precision: 0.93
Recall: 0.95


#### How often we encountered out of vocab words (Q2)

In [18]:
def find_vocab_proportions(num_unique_words_not_in_vocab, num_unique_words_in_vocab, num_test_words_in_vocab, num_test_words_total):
    # Proportion of unique words in the test set that were not in the vocab list
    prop_unique_not_in_vocab = num_unique_words_not_in_vocab / (num_unique_words_not_in_vocab + num_unique_words_in_vocab)
    prop_unique_in_vocab = 1 - prop_unique_not_in_vocab

    print(f"Proportion of unique words in vocab: {prop_unique_in_vocab}")
    print(f"Proportion of unique words not in vocab: {prop_unique_not_in_vocab}")

    # Proportion of words in the test set that were not in the vocab list
    prop_in_vocab = num_test_words_in_vocab / num_test_words_total
    prop_not_in_vocab = 1 - prop_in_vocab

    print(f"Proportion of test words in vocab: {prop_in_vocab}")
    print(f"Proportion of test words not in vocab: {prop_not_in_vocab}")

find_vocab_proportions(num_unique_words_not_in_vocab, num_unique_words_in_vocab, num_test_words_in_vocab, num_test_words_total)


Proportion of unique words in vocab: 0.9427178549664839
Proportion of unique words not in vocab: 0.05728214503351615
Proportion of test words in vocab: 0.9836797957695113
Proportion of test words not in vocab: 0.016320204230488744


#### Confidence of Classification (Q3)

In [19]:
# For all test instances, divide the posterior likelihoods for each class

# Initialise empty lists to store the ratios
likelihood_list_c0 = []
likelihood_list_c1 =[]

# For each test instance, we compute the ratio of the posterior probabilities
for post_prob_0, post_prob_1 in zip(c0_posterior, c1_posterior):
    likelihood_list_c0.append(post_prob_0 / post_prob_1)
    likelihood_list_c1.append(post_prob_1 / post_prob_0)

# Find the 3 most confident classifications for c0 (non-malicious)
c0_indexes = np.argsort(likelihood_list_c0)[-3:][::-1]
c0_ratios = np.sort(likelihood_list_c0)[-3:][::-1]
instances_c0 = test_df['textPreprocessed'].iloc[c0_indexes]

print(c0_ratios)
print(instances_c0)


[9.13499445e+37 2.69038186e+29 3.18292454e+25]
341    time : rs. transaction number & & & & & & & & ...
223    ? ? ? ? .. .. u u u u , , ... ... ... ... say ...
969    . every & & & & & & ; ; ; ; ; ; lt lt lt # # #...
Name: textPreprocessed, dtype: object


In [20]:
# Find the 3 most confident classifications for c1 (malicious)
c1_indexes = np.argsort(likelihood_list_c1)[-3:][::-1]
c1_ratios = np.sort(likelihood_list_c1)[-3:][::-1]
instances_c1 = test_df['textPreprocessed'].iloc[c1_indexes]

print(c1_ratios)
print(instances_c1)

[1.35388960e+20 1.28709054e+20 1.14912397e+20]
844    . 4 + call £ - * holiday & urgent 18 t landlin...
985    . 3 4 + ! call : £ offer * holiday & urgent 18...
460    . . . , please order text call / : customer to...
Name: textPreprocessed, dtype: object


In [21]:
# On the boundary between the two classes
# Find the 3 text instances with R values closest to 1
close_to_1_indexes = sorted(range(len(likelihood_list_c0)), key=lambda i: abs(likelihood_list_c0[i] - 1))[:3]
close_to_1_ratios = [likelihood_list_c0[i] for i in close_to_1_indexes]

instances_close_to_1 = test_df['textPreprocessed'].iloc[close_to_1_indexes]

print(close_to_1_ratios)
print(instances_close_to_1)


[1.017098135219966, 1.0441124738285466, 0.9297658433847519]
90                  . call dear
455                . reply glad
767    . . tell return re order
Name: textPreprocessed, dtype: object


## 3. Extending the model with semi-supervised training

### Active Learning

Split the labelled data into a training and validation set

In [22]:
# Original random split
# Random sample
sms_df_train = sms_df.sample(frac=0.8, random_state=1462474).reset_index(drop=True)

# Create the validation set
sms_df_validation = sms_df.drop(sms_df_train.index)


# Used when testing random samples with stratified split
"""
# Perform stratified split
sms_df_train, sms_df_validation = train_test_split(sms_df, test_size=0.2, random_state=1462474, stratify=sms_df['class'])"""


"\n# Perform stratified split\nsms_df_train, sms_df_validation = train_test_split(sms_df, test_size=0.2, random_state=1462474, stratify=sms_df['class'])"

We must read in the unlabelled data

In [23]:
sms_unlabelled = pd.read_csv('sms_unlabelled.csv')

In [24]:
# First we select 200 instances by random
sms_ul_sample = sms_unlabelled.sample(200, random_state=1462474)

# Append the sampled instances to the training set
sms_df_train = pd.concat([sms_df_train, sms_ul_sample], ignore_index=True)

In [25]:
# Preprocess the data
sms_df_train = preprocess_data(sms_df_train)
sms_df_validation = preprocess_data(sms_df_validation)

# For selecting predictive words later
sms_unlabelled = preprocess_data(sms_unlabelled)


Retrain our model on the new training set

In [26]:
# Retrains our model by calculating the likelihoods and priors of the new training set
def retrain_model(sms_df_train, alpha_new):
    # Retrain our model
    new_vocab_list = create_vocab_list(sms_df_train)

    # Build count matrix
    new_count_matrix, new_vocab_dict = build_count_matrix(sms_df_train, new_vocab_list)

    # Calculate priors
    priors = calculate_priors(sms_df_train)
    p_c0_new = priors[0]
    p_c1_new = priors[1]
    print(priors)

    # We need to find the number of times each word appears in each class
    count_c0_new = count_words_in_class(0,sms_df_train, new_count_matrix)
    count_c1_new = count_words_in_class(1,sms_df_train, new_count_matrix)
    print(sum(count_c0_new))
    print(sum(count_c0))

    # Calculate likelihoods using laplace smoothing
    class_0_c_new, class_1_c_new = calculate_likelihoods(count_c0_new, count_c1_new, new_vocab_dict, alpha_new)

    return class_0_c_new, class_1_c_new, new_vocab_list, new_vocab_dict

# Retrain our model
alpha_new = 1
class_0_c_new, class_1_c_new, new_vocab_list, new_vocab_dict = retrain_model(sms_df_train, alpha_new)


[0.7966666666666666, 0.20333333333333334]
14444.0
15861.0


#### Model is trained now we must evaluate it on our validation set

In [27]:
# Evaluates our model on the validation set by predicting the labels of each test instance
# and calculating the accuracy, confusion matrix, precision and recall
def validation_eval(true_labels, class_0_c_new, class_1_c_new, validation_data, new_vocab_list, new_vocab_dict):

    # Build the count matrix for the validation set
    new_test_count_matrix, num_test_words_in_vocab, num_test_words_not_in_vocab, unique_words_in_vocab, unique_words_not_in_vocab = build_test_count_matrix(validation_data, new_vocab_list, new_vocab_dict)

    # Count the number of unique words and in the vocab list and test (for question 2)
    num_unique_words_in_vocab = len(unique_words_in_vocab)
    num_unique_words_not_in_vocab = len(unique_words_not_in_vocab)
    num_test_words_total = num_test_words_in_vocab + num_test_words_not_in_vocab

    print(num_unique_words_not_in_vocab, num_unique_words_in_vocab, num_test_words_in_vocab, num_test_words_total)

    # Class 0 posterior probabilities
    log_c0_post, c0_posterior = calc_posterior_log(new_test_count_matrix, class_0_c_new, 0, priors)

    # Class 1 posterior probabilities
    log_c1_post, c1_posterior = calc_posterior_log(new_test_count_matrix, class_1_c_new, 1, priors)

    # Predict the labels
    pred_labels, probs = predict_nb(log_c0_post, log_c1_post)

    # Evaluate the model
    model_eval(true_labels, pred_labels)

    return c0_posterior, c1_posterior


# Call the function and evaluate the model
_,_ = validation_eval(sms_df_validation['class'].values, class_0_c_new, class_1_c_new, sms_df_validation, new_vocab_list, new_vocab_dict)


14 1115 4586 4602
Accuracy: 0.965
Confusion Matrix:
 [[309   9]
 [  5  77]]
Precision: 0.90
Recall: 0.94


#### Improving our model: Part 1

In [28]:
# First we must look at the 200 instances added at random to our training set

# Using the R value method we used earlier we will find the most predictive words
# from our unlabelled data and select the most predictive instances

# We will calculate the R value for each instance in the unlabelled data using our original trained model to predict posterior probabilities
def unlabelled_predictiveness(sms_unlabelled, vocab_list, vocab_dict, likelihood_lists, priors, n):

    # Create a count matrix for the unlabelled data
    unlabelled_count_matrix, num_ul_words_in_vocab, num_ul_words_not_in_vocab, unique_words_in_vocab, unique_words_not_in_vocab = build_test_count_matrix(sms_unlabelled, vocab_list, vocab_dict)


    # Find the posterior probabilities for each class
    log_ul_posteriors_0, unlabelled_class_0_posteriors = calc_posterior_log(unlabelled_count_matrix, likelihood_lists[0], 0, priors)
    log_ul_posteriors_1, unlabelled_class_1_posteriors = calc_posterior_log(unlabelled_count_matrix, likelihood_lists[1], 1, priors)


    # Calculate the R value for each instance

    # Initialise empty lists to store the R values
    ul_r_0 = []
    ul_r_1 = []

    for i in range(len(unlabelled_count_matrix)):
        # Calculate the R values using log operations
        log_r_0 = log_ul_posteriors_0[i] - log_ul_posteriors_1[i]
        log_r_1 = log_ul_posteriors_1[i] - log_ul_posteriors_0[i]

        # Append the R values
        ul_r_0.append(log_r_0)
        ul_r_1.append(log_r_1)

    # Choosing the highest R values didn't work so we will choose the R values closest to 1
    close_to_1_indexes_ul = sorted(range(len(ul_r_0)), key=lambda i: abs(ul_r_0[i] - 1))[:n]
    close_to_1_ratios_ul = [ul_r_0[i] for i in close_to_1_indexes_ul]
    instances_close_to_1 = sms_unlabelled.iloc[close_to_1_indexes_ul]

    # Try entropies instead
    # Entropy list
    entropies =[]

    for log_c_0_post, log_c_1_post in zip(log_ul_posteriors_0, log_ul_posteriors_1):

        # Convert to probabilities for entropy calculation
        c_0_post = np.exp(log_c_0_post)
        c_1_post = np.exp(log_c_1_post)

        # Ensure probabilities are normalised
        total_post = c_0_post + c_1_post
        c_0_post= c_0_post / total_post
        c_1_post = c_1_post / total_post

        # Calculate entropy
        entropy = - (c_0_post * np.log2(c_0_post) + c_1_post * np.log2(c_1_post))
        entropies.append(entropy)

    # Select the n instances with the highest entropy (most uncertain)
    highest_entropy_indexes = np.argsort(entropies)[-n:][::-1]
    instances_high_entropy = sms_unlabelled.iloc[highest_entropy_indexes]

    # We are using high entropy as it performs slightly better than the R value method
    return instances_high_entropy



# Find the 200 most predictive instances
n = 200
# Use the original trained model and priors to predict the posterior probabilities and get the predictive instances
instances_to_add = unlabelled_predictiveness(sms_unlabelled, vocab_list, vocab_dict, [class_0_c, class_1_c], priors, n)


#### Retrain model using these 200 words instead

#### Try sk learn train_test_split

In [29]:
from sklearn.model_selection import train_test_split

In [30]:
# Add the new instances to the training set
sms_df = pd.read_csv("sms_supervised_train.csv")

# Split the labelled data into a training and validation set with stratification
sms_df_improved_train, sms_df_improved_validation = train_test_split(sms_df, test_size=0.2, random_state=1462474, stratify=sms_df['class'] )

# Upsample uncertain records by duplicating them (not utilised in final results (as seen by * 1, just experimenting)
upsampled_instances = pd.concat([instances_to_add] * 1, ignore_index=True)

# Append the upsampled uncertain records to the training set
sms_df_improved_train = pd.concat([sms_df_improved_train, upsampled_instances], ignore_index=True)
sms_df_improved_train = sms_df_improved_train.reset_index(drop=True)

# Preprocess the data
sms_df_improved_train = preprocess_data(sms_df_improved_train)
sms_df_improved_validation = preprocess_data(sms_df_improved_validation)

# Retrain the model with the updated training set

alpha_improved = 1
class_0_c_new_i, class_1_c_new_i, new_vocab_list_i, new_vocab_dict_i = retrain_model(sms_df_improved_train, alpha_improved)

# Evaluate the model
c0_posteriors_i, c1_posteriors_i = validation_eval(
    sms_df_improved_validation['class'].values,
    class_0_c_new_i,
    class_1_c_new_i,
    sms_df_improved_validation,
    new_vocab_list_i,
    new_vocab_dict_i
)

[0.8033333333333333, 0.19666666666666666]
13473.0
15861.0
71 1063 4481 4577
Accuracy: 0.9825
Confusion Matrix:
 [[318   2]
 [  5  75]]
Precision: 0.97
Recall: 0.94


## 4. Supervised model evaluation

### Test model on test data

In [31]:
# Evaluate our model on the test set
_, _ = validation_eval(test_df['class'].values, class_0_c_new_i,
                class_1_c_new_i, test_df, new_vocab_list_i, new_vocab_dict_i)



133 1508 10716 10968
Accuracy: 0.977
Confusion Matrix:
 [[785  15]
 [  8 192]]
Precision: 0.93
Recall: 0.96


### Model's Representation

In [32]:
# Compare model confidence before and after semi-supervised training
def compare_model_confidence(c0_posterior_before, c1_posterior_before, c0_posterior_after, c1_posterior_after):
    # Calculate confidence ratios before and after training
    confidence_ratios_before = []
    confidence_ratios_after = []

    for (c0, c1) in zip(c0_posterior_before, c1_posterior_before):
        c0_c1 = c0 / c1
        c1_c0 = c1 / c0
        confidence_ratios_before.append([c0_c1, c1_c0])

    for (c0, c1) in zip(c0_posterior_after, c1_posterior_after):
        c0_c1 = c0 / c1
        c1_c0 = c1 / c0
        confidence_ratios_after.append([c0_c1, c1_c0])


    # Calculate and print average confidence for each class
    avg_confidence_before_c0_c1 = np.mean([ratio[0] for ratio in confidence_ratios_before])
    avg_confidence_before_c1_c0 = np.mean([ratio[1] for ratio in confidence_ratios_before])
    avg_confidence_after_c0_c1 = np.mean([ratio[0] for ratio in confidence_ratios_after])
    avg_confidence_after_c1_c0 = np.mean([ratio[1] for ratio in confidence_ratios_after])

    print(f"Average confidence before training (c0/c1): {avg_confidence_before_c0_c1:.4e}")
    print(f"Average confidence before training (c1/c0): {avg_confidence_before_c1_c0:.4e}")
    print(f"Average confidence after training (c0/c1): {avg_confidence_after_c0_c1:.4e}")
    print(f"Average confidence after training (c1/c0): {avg_confidence_after_c1_c0:.4e}")



# Compare most predictive words before and after our active learning
def compare_most_predictive_words(vocab_list_before, class_0_c_before, class_1_c_before,
                                   vocab_list_after, class_0_c_after, class_1_c_after, n=10):
    # Convert lists to NumPy arrays
    class_0_c_before = np.array(class_0_c_before)
    class_1_c_before = np.array(class_1_c_before)
    class_0_c_after = np.array(class_0_c_after)
    class_1_c_after = np.array(class_1_c_after)

    # Find the most predictive words before training (ratio of likelihoods)
    most_predictive_words_c0_before = [(vocab_list_before[i], (class_0_c_before[i] / class_1_c_before[i])) for i in np.argsort(class_0_c_before / class_1_c_before)[-n:][::-1]]
    most_predictive_words_c1_before = [(vocab_list_before[i], (class_1_c_before[i] / class_0_c_before[i])) for i in np.argsort(class_1_c_before / class_0_c_before)[-n:][::-1]]

    # Find the most predictive words after training
    most_predictive_words_c0_after = [(vocab_list_after[i], (class_0_c_after[i] / class_1_c_after[i])) for i in np.argsort(class_0_c_after / class_1_c_after)[-n:][::-1]]
    most_predictive_words_c1_after = [(vocab_list_after[i], (class_1_c_after[i] / class_0_c_after[i])) for i in np.argsort(class_1_c_after / class_0_c_after)[-n:][::-1]]

    # Print the results
    print("Most predictive words for class 0 before training with ratios:")
    for word, ratio in most_predictive_words_c0_before:
        print(f"{word}: {ratio}")

    print("\nMost predictive words for class 0 after training with ratios:")
    for word, ratio in most_predictive_words_c0_after:
        print(f"{word}: {ratio}")

    print("\nMost predictive words for class 1 before training with ratios:")
    for word, ratio in most_predictive_words_c1_before:
        print(f"{word}: {ratio}")

    print("\nMost predictive words for class 1 after training with ratios:")
    for word, ratio in most_predictive_words_c1_after:
        print(f"{word}: {ratio}")



# Call the functions
compare_model_confidence(c0_posterior, c1_posterior, c0_posteriors_i, c1_posteriors_i)
compare_most_predictive_words(vocab_list, class_0_c, class_1_c, new_vocab_list_i, class_0_c_new_i, class_1_c_new_i)

Average confidence before training (c0/c1): 9.1350e+34
Average confidence before training (c1/c0): 5.9150e+17
Average confidence after training (c0/c1): 3.2162e+17
Average confidence after training (c1/c0): 7.7935e+14
Most predictive words for class 0 before training with ratios:
;: 60.49921647638236
...: 57.4957092754272
gt: 54.06312961719275
lt: 53.548242668457576
:): 47.88448623237072
ü: 31.922990821580477
lor: 28.833669129169465
hope: 24.714573539288114
ok: 24.714573539288114
d: 21.110364898141928

Most predictive words for class 0 after training with ratios:
gt: 49.38611165142968
lt: 48.86072748492511
:): 36.77689165531998
...: 28.68597549114958
ü: 27.845360824742265
;: 27.582668741489982
lor: 23.116903326201125
ok: 17.863061661155417
hope: 17.863061661155417
d: 17.863061661155417

Most predictive words for class 1 before training with ratios:
prize: 99.05086956521738
tone: 64.09173913043477
£: 49.71965217391304
select: 46.61217391304348
claim: 45.96478260869565
paytm: 36.90130434